In [2]:
# import needed packages
import pandas as pd
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
import os

In [3]:
# data extraction function
try:
    data = pd.read_csv(r'zipco_transaction.csv')
    print("Data extraction successful.")
except Exception as e:
    print(f"Data extraction failed: {e}")

Data extraction successful.


In [4]:
data.head()

,Date,ProductName,Quantity,UnitPrice,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,...,DeliveryTime_min,OrderType,CustomerName,CustomerAddress,Customer_PhoneNumber,CustomerEmail,Staff_Name,Staff_Email,DayOfWeek,TotalSales
0,2023-01-01 00:00:00,Vanilla Cake,2,12.532304,South,Cash,True,Rainy,20.654914,Poor,...,30,In Store,William Adams,"9851 David Green\r\nTonyaburgh, VA 02853",(916)427-7276x861,lisa00@example.net,John Bridges,pdavidson@example.com,Sunday,25.064608
1,2023-01-01 01:00:00,Red Velvet Cake,1,7.083070,South,Cash,False,Rainy,23.549497,Average,...,33,In Store,Anthony Wiggins,"24682 Holly Stravenue\r\nMooreville, NH 13901",769.318.4373,michellefernandez@example.com,Sarah Bentley,ajohnson@example.net,Sunday,7.083070
2,2023-01-01 02:00:00,Chocolate Cake,5,6.736064,North,Cash,True,Rainy,NaN,Excellent,...,43,Phone Order,Ashley Duke,10184 Washington Trace Apt. 679\r\nEast Brandi...,702.520.3286,cooperwilliam@example.com,Connie Cervantes,michele29@example.net,Sunday,33.680321
3,2023-01-01 03:00:00,Carrot Cake,2,7.314823,North,Cash,False,Cloudy,20.137483,Poor,...,32,Online Order,Brandon Taylor,"87194 Jeff Rue\r\nMitchellbury, CA 50463",622-527-9530,fsilva@example.net,Jessica Stewart,xwilson@example.org,Sunday,14.629647
4,2023-01-01 04:00:00,Pizza Pepperoni,1,7.577727,East,Credit Card,True,Cloudy,23.020987,Good,...,58,In Store,Brittany Watkins,"850 Julia Groves\r\nHartview, WI 95954",759-517-8359,petersjoseph@example.net,Cheryl Carpenter,christine96@example.org,Sunday,7.577727


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1005 non-null   object 
 1   ProductName             1005 non-null   object 
 2   Quantity                1005 non-null   int64  
 3   UnitPrice               1005 non-null   float64
 4   StoreLocation           1005 non-null   object 
 5   PaymentType             1005 non-null   object 
 6   PromotionApplied        1005 non-null   bool   
 7   Weather                 1005 non-null   object 
 8   Temperature             904 non-null    float64
 9   StaffPerformanceRating  1005 non-null   object 
 10  CustomerFeedback        905 non-null    object 
 11  DeliveryTime_min        1005 non-null   int64  
 12  OrderType               1005 non-null   object 
 13  CustomerName            1005 non-null   object 
 14  CustomerAddress         1005 non-null   

In [6]:
# data transformation and cleaning

# remove duplicates entry

data.drop_duplicates(inplace=True)


In [7]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             900 non-null    float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        900 non-null    object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [8]:
# set code to go through column to replace missing values 
numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
for col in numeric_cols:
    #mean_value = data[col].mean()
    data.fillna({col: data[col].mean()}, inplace=True)

In [29]:
data['Date'] = pd.to_datetime(data['Date'])

In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Date                    1000 non-null   datetime64[ns]
 1   ProductName             1000 non-null   object        
 2   Quantity                1000 non-null   int64         
 3   UnitPrice               1000 non-null   float64       
 4   StoreLocation           1000 non-null   object        
 5   PaymentType             1000 non-null   object        
 6   PromotionApplied        1000 non-null   bool          
 7   Weather                 1000 non-null   object        
 8   Temperature             1000 non-null   float64       
 9   StaffPerformanceRating  1000 non-null   object        
 10  CustomerFeedback        1000 non-null   object        
 11  DeliveryTime_min        1000 non-null   int64         
 12  OrderType               1000 non-null   object        

In [10]:
#fill with unknown for string column
string_cols = data.select_dtypes(include=['object']).columns
for col in string_cols:
    data.fillna({col: 'Unknown'}, inplace=True)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    1000 non-null   object 
 1   ProductName             1000 non-null   object 
 2   Quantity                1000 non-null   int64  
 3   UnitPrice               1000 non-null   float64
 4   StoreLocation           1000 non-null   object 
 5   PaymentType             1000 non-null   object 
 6   PromotionApplied        1000 non-null   bool   
 7   Weather                 1000 non-null   object 
 8   Temperature             1000 non-null   float64
 9   StaffPerformanceRating  1000 non-null   object 
 10  CustomerFeedback        1000 non-null   object 
 11  DeliveryTime_min        1000 non-null   int64  
 12  OrderType               1000 non-null   object 
 13  CustomerName            1000 non-null   object 
 14  CustomerAddress         1000 non-null   object

In [16]:
#create product table
products = data[['ProductName', 'UnitPrice']].drop_duplicates().reset_index(drop=True)
products.index.name = 'ProductID'
products = products.reset_index()

In [17]:
products.head(10)

,ProductID,ProductName,UnitPrice
0,0,Vanilla Cake,12.532304
1,1,Red Velvet Cake,7.083070
2,2,Chocolate Cake,6.736064
3,3,Carrot Cake,7.314823
4,4,Pizza Pepperoni,7.577727
5,5,Red Velvet Cake,14.568156
6,6,Red Velvet Cake,16.114719
7,7,Carrot Cake,12.949873
8,8,Carrot Cake,17.435266
9,9,Pizza Margherita,19.868948


In [18]:
# create customer table
customers = data[['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail']].drop_duplicates().reset_index(drop=True)
customers.index.name = 'CustomerID'
customers = customers.reset_index()

In [20]:
customers.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   CustomerID            1000 non-null   int64 
 1   CustomerName          1000 non-null   object
 2   CustomerAddress       1000 non-null   object
 3   Customer_PhoneNumber  1000 non-null   object
 4   CustomerEmail         1000 non-null   object
dtypes: int64(1), object(4)
memory usage: 39.2+ KB


In [22]:
# staff table
staff = data[['Staff_Name', 'Staff_Email']].drop_duplicates().reset_index(drop=True)
staff.index.name = 'StaffID'
staff = staff.reset_index()

In [ ]:
# rename data table to transaction table

,StaffID,Staff_Name,Staff_Email
0,0,John Bridges,pdavidson@example.com
1,1,Sarah Bentley,ajohnson@example.net
2,2,Connie Cervantes,michele29@example.net
3,3,Jessica Stewart,xwilson@example.org
4,4,Cheryl Carpenter,christine96@example.org


In [24]:
#transaction table
transactions = data.merge(products, on=['ProductName', 'UnitPrice'], how='left') \
                   .merge(customers, on=['CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail'], how='left') \
                   .merge(staff, left_on=['Staff_Name', 'Staff_Email'], right_on=['Staff_Name', 'Staff_Email'], how='left')

transactions.index.name = 'TransactionID'
transactions = transactions.reset_index() \
                     .drop(columns=['ProductName', 'UnitPrice', 'CustomerName', 'CustomerAddress', 'Customer_PhoneNumber', 'CustomerEmail', 'Staff_Name', 'Staff_Email'])   


In [25]:
transactions.head()

,TransactionID,Date,Quantity,StoreLocation,PaymentType,PromotionApplied,Weather,Temperature,StaffPerformanceRating,CustomerFeedback,DeliveryTime_min,OrderType,DayOfWeek,TotalSales,ProductID,CustomerID,StaffID
0,0,2023-01-01 00:00:00,2,South,Cash,True,Rainy,20.654914,Poor,Neutral,30,In Store,Sunday,25.064608,0,0,0
1,1,2023-01-01 01:00:00,1,South,Cash,False,Rainy,23.549497,Average,Positive,33,In Store,Sunday,7.083070,1,1,1
2,2,2023-01-01 02:00:00,5,North,Cash,True,Rainy,27.154342,Excellent,Unknown,43,Phone Order,Sunday,33.680321,2,2,2
3,3,2023-01-01 03:00:00,2,North,Cash,False,Cloudy,20.137483,Poor,Positive,32,Online Order,Sunday,14.629647,3,3,3
4,4,2023-01-01 04:00:00,1,East,Credit Card,True,Cloudy,23.020987,Good,Neutral,58,In Store,Sunday,7.577727,4,4,4


In [32]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   TransactionID           1000 non-null   int64         
 1   Date                    1000 non-null   datetime64[ns]
 2   Quantity                1000 non-null   int64         
 3   StoreLocation           1000 non-null   object        
 4   PaymentType             1000 non-null   object        
 5   PromotionApplied        1000 non-null   bool          
 6   Weather                 1000 non-null   object        
 7   Temperature             1000 non-null   float64       
 8   StaffPerformanceRating  1000 non-null   object        
 9   CustomerFeedback        1000 non-null   object        
 10  DeliveryTime_min        1000 non-null   int64         
 11  OrderType               1000 non-null   object        
 12  DayOfWeek               1000 non-null   object   

In [33]:
# save data to CV
data.to_csv('cleaned_data.csv', index=False)
products.to_csv('products.csv', index=False)
customers.to_csv('customers.csv', index=False)
staff.to_csv('staff.csv', index=False)
transactions.to_csv('transactions.csv', index=False)
print("Data saved to CSV files.")

Data saved to CSV files.
